In [0]:
!ls

In [0]:
import cv2
import numpy as np
import csv
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D 
from keras.layers import Lambda, Cropping2D, Dropout, ELU
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd

In [0]:
#find versions
import sys
import tensorflow

print (sys.version)
print(np.version.version)
print(tensorflow.__version__)
print(keras.__version__)

In [0]:
#load lines in csv file to read the images #load l 
def load_csv(file):
    lines = []
    with open(file) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            lines.append(line)
    return lines[1:]

In [0]:
file = 'Behavioral-Cloning/data/driving_log.csv'
image_path = 'Behavioral-Cloning/data/IMG/'

In [0]:
#load image names in the csv file
lines_path = load_csv(file)

#split not the data just their names
# We don't need to test images because it is a regression problem not classification.
train_samples, validation_samples = train_test_split(lines_path, shuffle=True, test_size=0.2)

print(lines_path)

In [0]:
# generally load images in the file 
def load_images(lines_path, image_path):
    path = image_path 
    lines = lines_path
    images = []
    angles = []
    
    for line in lines:
        for i in range(3):
            source_path = line[i]
            filename = source_path.split('/')[-1]
            current_path = path+filename
            image = cv2.imread(current_path)
            images.append(image)
            angle = float(line[3])
            if i == 0:
                angles.append(angle)
            elif i == 1:
                angles.append(angle + 0.20)
            else:
                angles.append(angle - 0.20)
    X_train = np.array(images)
    y_train = np.array(angles) 
    return X_train, y_train

In [0]:
import time
t0 = time.time()
X_train_normal, y_train_normal = load_images(train_samples, image_path)
X_valid_normal, y_valid_normal = load_images(validation_samples, image_path)
t1=time.time()
print("total seconds for loaded: {} sn".format(round(t1-t0)))

In [0]:
def resize(img):
    return tensorflow.image.resize_images(img, (60, 120))

In [0]:
model = Sequential()

# Crop 70 pixels from the top of the image and 25 from the bottom
model.add(Cropping2D(cropping=((75, 25), (0, 0)),
                     input_shape=(160, 320, 3),
                     data_format="channels_last"))

# Resize the data
model.add(Lambda(resize))

# Normalize the data
model.add(Lambda(lambda x: (x/127.5) - 0.5))

model.add(Conv2D(3, (1, 1), padding='same'))
model.add(ELU())

model.add(BatchNormalization())
model.add(Conv2D(16, (5, 5), strides=(2, 2), padding="same"))
model.add(ELU())

model.add(BatchNormalization())
model.add(Conv2D(32, (5, 5), strides=(2, 2), padding="same"))
model.add(ELU())

model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), strides=(2, 2), padding="same"))
model.add(ELU())

model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), strides=(2, 2), padding="same"))
model.add(ELU())

model.add(Flatten())
model.add(ELU())

model.add(Dense(512))
model.add(Dropout(.2))
model.add(ELU())

model.add(Dense(100))
model.add(Dropout(.5))
model.add(ELU())

model.add(Dense(10))
model.add(Dropout(.5))
model.add(ELU())

model.add(Dense(1))

adam = Adam(lr=1e-5)
model.compile(optimizer= adam, loss="mse", metrics=['accuracy'])

#Showing model and params, it is a heavy network with nearly 2.2 bilion params
model.summary()

In [0]:
results = model.fit(X_train_normal, y_train_normal,
                    validation_data=(X_valid_normal, y_valid_normal),
                    batch_size=32, epochs=1, verbose=1)

In [0]:
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('Loss')
plt.xlabel('epoch')

In [0]:
model.save('new_model.h5')

In [0]:
from google.colab import files
files.download('new_model.h5')